In [82]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [83]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN
2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN
3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN
4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN


In [85]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,1229 RICHARDSON HWY,DELTA JUNCTION,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",1229 Richardson Hwy,DELTA JUNCTION,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,1706 Ship Ave,Anchorage,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,46877 STILLPOINT TRL,HALIBUT COVE,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,1706 SHIP AVENUE,ANCHORAGE,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968552,968552,978144,977196,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,...,999 Other,6212 Offices of Dentists,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",NaN,NaN,NaN
968553,968553,978145,977197,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,...,999 Other,6244 Child Day Care Services,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",NaN,NaN,NaN
968554,968554,978146,977198,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,...,999 Other,2382 Building Equipment Contractors,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",NaN,NaN,NaN
968555,968555,978147,977199,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,...,999 Other,6216 Home Health Care Services,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",NaN,NaN,NaN


In [86]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
FL    60040
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
IN    16601
MO    16350
LA    13668
OR    13456
UT     9295
IA     8295
NE     5804
HI     4980
NM     4485
ID     4416
ND     3222
MT     3144
PR     2952
SD     2695
OK     2535
WY     2139
SC     2045
MS     1589
NV     1569
KS     1485
GU      443
RI      414
VI      290
MP       82
NC       11
AL        8
PA        8
AK        5
AZ        4
NJ        3
MA        3
TN        3
AR        2
IL        2
ME        2
GA        1
OH        1
CT        1
WV        1
Name: BorrowerState, dtype: int64

In [5]:
ppp_togeoMS = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "MS"]
ppp_togeoMS #5502

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
516299,516299,3506337207,4/27/20,470,PPP,STAFF PRO LLC,238 COURTHOUSE RD,GULFPORT,MS,39507.0,...,999 Other,5613 Employment Services,5613.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"238 COURTHOUSE RD, GULFPORT, MS",NaN,NaN,NaN
516300,516300,8685327005,4/8/20,470,PPP,"BUTLER SNOW, LLP",1020 HIGHLAND COLONY PKWY,RIDGELAND,MS,39157.0,...,999 Other,5411 Legal Services,5411.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",NaN,NaN,NaN
516301,516301,5587558700,4/2/21,470,PPP,MISSISSIPPI METHODIST SENIOR SERVICES IN,109 S Broadway St,Tupelo,MS,38804.0,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"109 S Broadway St, Tupelo, MS",NaN,NaN,NaN
516302,516302,8446217010,4/8/20,470,PPP,SOUTHERN ELECTRIC CORPORATION OF MISSISSIPPI,4374-A MANGUM DR,FLOWOOD,MS,39232.0,...,999 Other,2371 Utility System Construction,2371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"4374-A MANGUM DR, FLOWOOD, MS",NaN,NaN,NaN
516303,516303,3918387109,4/12/20,470,PPP,FRANKLIN CORPORATION,600 FRANKLIN DR,HOUSTON,MS,38851.0,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"600 FRANKLIN DR, HOUSTON, MS",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521796,521796,6662018610,3/23/21,470,PPS,GATHRIGHT-REED MEDICAL SUPPLY,300 Heritage Dr,Oxford,MS,38655.0,...,999 Other,4234 Professional and Commercial Equipment and...,4234.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"300 Heritage Dr, Oxford, MS",NaN,NaN,NaN
521797,521797,7192318409,2/11/21,470,PPS,FULWOOD CONSTRUCTION COMPANY LLC,6229 Highway 305 N Ste O,Olive Branch,MS,38654.0,...,999 Other,5324 Commercial and Industrial Machinery and E...,5324.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6229 Highway 305 N Ste O, Olive Branch, MS",NaN,NaN,NaN
521798,521798,7483017308,4/30/20,470,PPP,"S & J TRANSFER & STORAGE, LTD",41033 OLD HIGHWAY 45 South,ABERDEEN,MS,39730.0,...,999 Other,4931 Warehousing and Storage,4931.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",NaN,NaN,NaN
521799,521799,7726238600,3/24/21,470,PPS,OTIS LOGGING INC.,116 Price St,McComb,MS,39648.0,...,998 All Other Miscellaneous Manufacturing,1133 Logging,1133.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"116 Price St, McComb, MS",NaN,NaN,NaN


In [6]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoMS_add = ppp_togeoMS[cols]
ppp_togeoMS_add

,LoanNumber,full_add
516299,3506337207,"238 COURTHOUSE RD, GULFPORT, MS"
516300,8685327005,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS"
516301,5587558700,"109 S Broadway St, Tupelo, MS"
516302,8446217010,"4374-A MANGUM DR, FLOWOOD, MS"
516303,3918387109,"600 FRANKLIN DR, HOUSTON, MS"
...,...,...
521796,6662018610,"300 Heritage Dr, Oxford, MS"
521797,7192318409,"6229 Highway 305 N Ste O, Olive Branch, MS"
521798,7483017308,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS"
521799,7726238600,"116 Price St, McComb, MS"


In [8]:
add = ppp_togeoMS["full_add"].astype(str).to_list()

In [9]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('28')] #MS
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


125 South Congress St. Suite 1100, JACKSON, MS is Not Found
800 HIGHWAY 98 BYP, COLUMBIA, MS is Not Found
100 CATERPILLAR DR, FLOWOOD, MS is Not Found
530 SOLDIER COLONY RD, CANTON, MS is Not Found
101 MAX PL, FULTON, MS is Not Found
2510 LAKELAND DR, FLOWOOD, MS is Not Found
415 HIGHWAY 49 S, RICHLAND, MS is Not Found
329 MOSELLE SEMINARY RD, MOSELLE, MS is Not Found
1001 HOLLAND AVE, PHILADELPHIA, MS is Not Found
605 CRESCENT BLVD, RIDGELAND, MS is Not Found
373 HIGHWAY 49 S, RICHLAND, MS is Not Found
146 FURNITURE DR, NETTLETON, MS is Not Found
969 Lakeland Drive, JACKSON, MS is Not Found
711 AVIGNON DR, RIDGELAND, MS is Not Found
13491 Hwy 28 West, TAYLORSVILLE, MS is Not Found
1505 HIGHWAY 82, GREENWOOD, MS is Not Found
441 HALEY BARBOUR PKWY, YAZOO CITY, MS is Not Found
PO BOX 397, ROSEDALE, MS is Not Found
308 CORPORATE DR, RIDGELAND, MS is Not Found
1312 INDUSTRIAL ACCESS RD, WEST POINT, MS is Not Found
299 SOUTH ST, ISOLA, MS is Not Found
908 STATE HIGHWAY 15, NEW ALBANY, MS i

265 Quality Drive, BYHALIA, MS is Not Found
95 PLEASANT HILL RD, NESBIT, MS is Not Found
307 WALKER CIR, RICHLAND, MS is Not Found
168 ROSECREST DR, COLUMBUS, MS is Not Found
5625 HIGHWAY 18 W, JACKSON, MS is Not Found
5103 PARK ST, MOSS POINT, MS is Not Found
4554 State Highway 12 East, BELZONI, MS is Not Found
1625 Eastover Dr, Jackson, MS is Not Found
2604 Sunset Dr, GRENADA, MS is Not Found
310 ENTERPRISE DR, OXFORD, MS is Not Found
69900 HIGHWAY 82, GREENWOOD, MS is Not Found
1500 Port st, Friars Point, MS is Not Found
100 RESEARCH BLVD, STARKVILLE, MS is Not Found
95 Pleasant Hill Rd, Nesbit, MS is Not Found
160 COMMERCE DR, BAY SPRINGS, MS is Not Found
4000 Lakeland 0.0, Flowood, MS is Not Found
801 AZALEA DR D, WAYNESBORO, MS is Not Found
8300 Highway 19 N, Collinsville, MS is Not Found
87 Pickering Road, COLLINS, MS is Not Found
260 HIGHWAY 468, PEARL, MS is Not Found
725, New Albany, MS is Not Found
5297 OLD HIGHWAY 11 Sutie 1, HATTIESBURG, MS is Not Found
971 LAKELAND DR, JA

15109 County Road 626,, Greenwood, MS is Not Found
6555 US HWY 98 WEST, SUITE 13, HATTIESBURG, MS is Not Found
605 2nd Ave N, Columbus, MS is Not Found
730 ALABAMA ST, COLUMBUS, MS is Not Found
1 Magnolia Dr, West Point, MS is Not Found
276 Nissan Pkwy # F100, Canton, MS is Not Found
5262 South Raymond Ave, VERONA, MS is Not Found
605 2ND AVE NORTH, COLUMBUS, MS is Not Found
SAM T BARKLEY DR, NEW ALBANY, MS is Not Found
1943 S MADISON ST, TUPELO, MS is Not Found
1091 Tommy Munro Dr, Biloxi, MS is Not Found
2102 VETERANS BLVD, MCCOMB, MS is Not Found
5002 Highway 39 N Bldg B, Meridian, MS is Not Found
160 Marshall Durbin Dr, Waynesboro, MS is Not Found
9140-C CANAL ROAD, GULFPORT, MS is Not Found
2344 S GREEN ST, TUPELO, MS is Not Found
124 One Madison Plaza  Suite 1500, MADISON, MS is Not Found
1000 Highland Colony Pkwy Ste 9015, Ridgeland, MS is Not Found
100 Mill Street, Jonestown, MS is Not Found
100 Mill Street, Jonestown, MS is Not Found
2001 AIRPORT RD STE 305, FLOWOOD, MS is Not

108 Lexington Dr, Madison, MS is Not Found
108 Lexington Drive, MADISON, MS is Not Found
200 Township Pl, Ridgeland, MS is Not Found
5440 HIGHWAY 11 N, ELLISVILLE, MS is Not Found
2456 HWY 145, SALTILLO, MS is Not Found
273 PO BOX, MATHISTON, MS is Not Found
1500 Port Road, FRIARS POINT, MS is Not Found
4500 I 55 N Ste 292, Jackson, MS is Not Found
1101 S BEASLEY RD, PHEBA, MS is Not Found
3959 HIGHWAY 29, NEW AUGUSTA, MS is Not Found
2005 Highway 82 W, Greenwood, MS is Not Found
2005 Highway 82 West, GREENWOOD, MS is Not Found
101 RAINBOW WAY, MCCOMB, MS is Not Found
410 industrial park, HOLLY SPRINGS, MS is Not Found
1290, LOUISVILLE, MS is Not Found
817 OLD HIGHWAY 471, BRANDON, MS is Not Found
4875 I 55 NORTH SUITE 210, JACKSON, MS is Not Found
4381 EASON BLVD, TUPELO, MS is Not Found
600 LEIGH DR, COLUMBUS, MS is Not Found
330 Green Wave Lane, BATESVILLE, MS is Not Found
3688 VETERANS MEMORIAL DR, HATTIESBURG, MS is Not Found
2301 Hwy 39 N, MERIDIAN, MS is Not Found
PO BOX 3089, O

406 INTERCHANGE DR, FULTON, MS is Not Found
715 Highway 82 West, GREENWOOD, MS is Not Found
1211 GLOSTER ST SUITE B, TUPELO, MS is Not Found
5260 North Church Ave P.O. Box 490, LOUISVILLE, MS is Not Found
4365 Highway 51 S, Senatobia, MS is Not Found
110 Cypress Center Dr, Vicksburg, MS is Not Found
3037 Ellisvillle Boulevard, LAUREL, MS is Not Found
503 FRONT STREET EXT, MERIDIAN, MS is Not Found
2000 Highway 39 N, Meridian, MS is Not Found
2000 HIGHWAY 39 North, MERIDIAN, MS is Not Found
2000 Byron St, Hattiesburg, MS is Not Found
513 Cobblestone Court, Madison, MS is Not Found
106, STARKVILLE, MS is Not Found
2170 HIGHWAY 51 STE 1, HERNANDO, MS is Not Found
971 Lakeland Dr Suite 401, Jackson, MS is Not Found
120 GEORGE AVE, PETAL, MS is Not Found
2725 Old Belden Circle Suite D, TUPELO, MS is Not Found
3151 GREENFIELD RD, PEARL, MS is Not Found
4500 Interstate 55 N Ste 109, JACKSON, MS is Not Found
4500 Interstate 55 N Suite 109, JACKSON, MS is Not Found
725, New Albany, MS is Not Fo

4780 I-55 N Suite 400, JACKSON, MS is Not Found
508 HIGHWAY 98 BYP, COLUMBIA, MS is Not Found
53 RIVERS DR, HATTIESBURG, MS is Not Found
1272 Evelyn Gandy Pkwy, PETAL, MS is Not Found
1201 Clinton Raymond Rd, Clinton, MS is Not Found
109 S Meridian St N/A, Aberdeen, MS is Not Found
100 POST OAK RD, MADISON, MS is Not Found
3204 Baldwin Road, GREENWOOD, MS is Not Found
575A Hwy 4W, BOONEVILLE, MS is Not Found
361 Township Ave Ste 200, Ridgeland, MS is Not Found
1300 Weyerhaeuser St, Philadelphia, MS is Not Found
41029 OLD HIGHWAY 45 South, ABERDEEN, MS is Not Found
115 Highway 24 E, CENTREVILLE, MS is Not Found
2601 Getwell Rd Ste 1, Corinth, MS is Not Found
2782 Starlanding East, Southaven, MS is Not Found
15429 HIGHWAY 49, BELZONI, MS is Not Found
15429 Highway 49, Belzoni, MS is Not Found
P O Box 1109, BAY SPRINGS, MS is Not Found
271 Quality Drive, BYHALIA, MS is Not Found
2829 LAKELAND DR Suite 1502, FLOWOOD, MS is Not Found
106, FLORA, MS is Not Found
115 Mayhew St, West Point, MS

100 CYPRESS CENTER DR, VICKSBURG, MS is Not Found
113 Red Hill Church Rd, Lumberton, MS is Not Found
12890 HIGHWAY 15 S, LOUISVILLE, MS is Not Found
71001 HIGHWAY 82 W, GREENWOOD, MS is Not Found
60 Technology blvd STE 105 A, Starkville, MS is Not Found
1601 HIGHWAY 82 W, GREENWOOD, MS is Not Found
4245 LAKELAND DR, FLOWOOD, MS is Not Found
3513 Highway 8 West, CLEVELAND, MS is Not Found
2110 HIGHWAY 80 Suite A, PEARL, MS is Not Found
1811 COUNTY ROAD 272, CARROLLTON, MS is Not Found
22019 HIGHWAY 45 N, ABERDEEN, MS is Not Found
1530 CLINTON BUSINESS PARK DR, CLINTON, MS is Not Found
16831 HIGHWAY 21 S, SEBASTOPOL, MS is Not Found
777 HIGHWAY 468, BRANDON, MS is Not Found
9335 Highway 19 N, Collinsville, MS is Not Found
650 PARK PLAZA DR, NEW ALBANY, MS is Not Found
971 Lakeland Drive Ste 1157, JACKSON, MS is Not Found
1949 HIGHWAY 80, FLOWOOD, MS is Not Found
1090 Willis Rd, Clarksdale, MS is Not Found
10040 Highway 371, Marietta, MS is Not Found
10040 HIGHWAY 371, MARIETTA, MS is Not

1023 Highway 39 N, Meridian, MS is Not Found
110 N Jerry Clower Blvd, Yazoo City, MS is Not Found
110 N JERRY CLOWER BLVD SUITE W, YAZOO CITY, MS is Not Found
PO BOX 0580, PEARLINGTON, MS is Not Found
1023 HIGHWAY 39 N, MERIDIAN, MS is Not Found
5247 GREENWAY DRIVE EXT, Jackson, MS is Not Found
215 NORTH ST 215 ST, PASS CHRISTIAN, MS is Not Found
130 Robert E Lee Dr, Lexington, MS is Not Found
5230 HIGHWAY 25, FLOWOOD, MS is Not Found
109 HIGHWAY 51, BROOKHAVEN, MS is Not Found
6820 Cobblestone Blvd Suite 3, SOUTHAVEN, MS is Not Found
9325 Highway 19 N Unit D, Collinsville, MS is Not Found
253 Ashmore Dr, Grenada, MS is Not Found
719 Air Industrial Park Rd, GRENADA, MS is Not Found
134 LEONTYNE PRICE BLVD, LAUREL, MS is Not Found
3301 TININ DRIVE, CORINTH, MS is Not Found
5719 Highway 25 Ste 104 5719 Highway 25 Suite 104, Flowood, MS is Not Found
130 ROBERT E LEE DR, LEXINGTON, MS is Not Found
699 SUNSET DR, GRENADA, MS is Not Found
20 East LINCOLN RD, BROOKHAVEN, MS is Not Found
2475 

24341 Graham Dr, TUPELO, MS is Not Found
6045 I 55, BYRAM, MS is Not Found
6045 I 55 S, Byram, MS is Not Found
2057 ATTALA ROAD 1223, KOSCIUSKO, MS is Not Found
905  22ND AVE UPPR 3, MERIDIAN, MS is Not Found
298 S Perkins Street, RIDGELAND, MS is Not Found
298 S Perkins St, Ridgeland, MS is Not Found
517 Highway 82 West, GREENWOOD, MS is Not Found
5 Pershing St, Belmont, MS is Not Found
5 PERSHING ST, BELMONT, MS is Not Found
128 Riverview Drive, FLOWOOD, MS is Not Found
718 SAUNDERS ST, MCCOMB, MS is Not Found
500 Killebrew Street, LYON, MS is Not Found
102 FIRST CHOICE DR, MADISON, MS is Not Found
499 GLOSTER CREEK STE. H11, TUPELO, MS is Not Found
2 Old River Pl Ste N, Jackson, MS is Not Found
274 Structural Lane, COLUMBUS, MS is Not Found
16829 Kapalama Rd, Pass Christian, MS is Not Found
2353 Highway 51 S Ste 1, Hernando, MS is Not Found
796 HIGHWAY 82 EAST, WINONA, MS is Not Found
2259 Graham Drive, TUPELO, MS is Not Found
PO Box 16521, JACKSON, MS is Not Found
302 HIGHLAND PARK

In [10]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

5502
5505
5502
5502


In [11]:
#pd.set_option('display.max_rows', None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df

,Lat
0,30.381123
1,32.446636
2,34.257282
3,32.31944
4,33.914646
...,...
5500,34.367306
5501,34.95232
5502,None
5503,31.24796


In [14]:
test_lat = lat_df.sort_values(by="Lat",ascending=True)
test_lat

,Lat
4290,11.97984
4610,11.98175
4613,11.98175
16,30.231115
153,30.231115
...,...
5489,None
5490,None
5497,None
5499,None


In [16]:
lat_df = lat_df.drop(index=[4290,4610,4613],axis=1)
lat_list = lat_df["Lat"].astype(str).to_list()

<b>Add remaining values (2801) to result_df(5461)

In [17]:
result_df = pd.DataFrame({"full_add":add, "Lat":lat_list, "Long":long, "FIPS_z": geo_id})
result_df 

,full_add,Lat,Long,FIPS_z
0,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044,[280470014022015]
1,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.15197,[280890302051012]
2,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,[280819507001030]
3,"4374-A MANGUM DR, FLOWOOD, MS",32.31944,-90.10765,[281210203041000]
4,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.01671,[280179503002008]
...,...,...,...,...
5497,"300 Heritage Dr, Oxford, MS",34.367306,-89.56031,[280719503042000]
5498,"6229 Highway 305 N Ste O, Olive Branch, MS",34.95232,-89.830414,[280330708213005]
5499,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",None,None,None
5500,"116 Price St, McComb, MS",31.24796,-90.46982,[281139504002077]


In [18]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

,full_add,Lat,Long,FIPS_z
0,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044,280470014022015
1,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.15197,280890302051012
2,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,280819507001030
3,"4374-A MANGUM DR, FLOWOOD, MS",32.31944,-90.10765,281210203041000
4,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.01671,280179503002008
...,...,...,...,...
5497,"300 Heritage Dr, Oxford, MS",34.367306,-89.56031,280719503042000
5498,"6229 Highway 305 N Ste O, Olive Branch, MS",34.95232,-89.830414,280330708213005
5499,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",None,None,None
5500,"116 Price St, McComb, MS",31.24796,-90.46982,281139504002077


In [39]:
ppp_togeoMS = ppp_togeoMS.reset_index()
ppp_togeoMS

,index,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,516299,516299,3506337207,4/27/20,470,PPP,STAFF PRO LLC,238 COURTHOUSE RD,GULFPORT,MS,...,999 Other,5613 Employment Services,5613.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"238 COURTHOUSE RD, GULFPORT, MS",NaN,NaN,NaN
1,516300,516300,8685327005,4/8/20,470,PPP,"BUTLER SNOW, LLP",1020 HIGHLAND COLONY PKWY,RIDGELAND,MS,...,999 Other,5411 Legal Services,5411.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",NaN,NaN,NaN
2,516301,516301,5587558700,4/2/21,470,PPP,MISSISSIPPI METHODIST SENIOR SERVICES IN,109 S Broadway St,Tupelo,MS,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"109 S Broadway St, Tupelo, MS",NaN,NaN,NaN
3,516302,516302,8446217010,4/8/20,470,PPP,SOUTHERN ELECTRIC CORPORATION OF MISSISSIPPI,4374-A MANGUM DR,FLOWOOD,MS,...,999 Other,2371 Utility System Construction,2371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"4374-A MANGUM DR, FLOWOOD, MS",NaN,NaN,NaN
4,516303,516303,3918387109,4/12/20,470,PPP,FRANKLIN CORPORATION,600 FRANKLIN DR,HOUSTON,MS,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"600 FRANKLIN DR, HOUSTON, MS",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5497,521796,521796,6662018610,3/23/21,470,PPS,GATHRIGHT-REED MEDICAL SUPPLY,300 Heritage Dr,Oxford,MS,...,999 Other,4234 Professional and Commercial Equipment and...,4234.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"300 Heritage Dr, Oxford, MS",NaN,NaN,NaN
5498,521797,521797,7192318409,2/11/21,470,PPS,FULWOOD CONSTRUCTION COMPANY LLC,6229 Highway 305 N Ste O,Olive Branch,MS,...,999 Other,5324 Commercial and Industrial Machinery and E...,5324.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6229 Highway 305 N Ste O, Olive Branch, MS",NaN,NaN,NaN
5499,521798,521798,7483017308,4/30/20,470,PPP,"S & J TRANSFER & STORAGE, LTD",41033 OLD HIGHWAY 45 South,ABERDEEN,MS,...,999 Other,4931 Warehousing and Storage,4931.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",NaN,NaN,NaN
5500,521799,521799,7726238600,3/24/21,470,PPS,OTIS LOGGING INC.,116 Price St,McComb,MS,...,998 All Other Miscellaneous Manufacturing,1133 Logging,1133.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"116 Price St, McComb, MS",NaN,NaN,NaN


In [48]:
#mapping coordinates to NaN values in main 
m = dict(zip(result_df.full_add,result_df.Lat))
ppp_togeoMS.Lat = ppp_togeoMS.Lat.fillna(ppp_togeoMS.full_add.map(m))
n = dict(zip(result_df.full_add,result_df.Long))
ppp_togeoMS.Long = ppp_togeoMS.Long.fillna(ppp_togeoMS.full_add.map(n))

In [49]:
ppp_togeoMS

,index,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,516299,516299,3506337207,4/27/20,470,PPP,STAFF PRO LLC,238 COURTHOUSE RD,GULFPORT,MS,...,999 Other,5613 Employment Services,5613.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044,NaN
1,516300,516300,8685327005,4/8/20,470,PPP,"BUTLER SNOW, LLP",1020 HIGHLAND COLONY PKWY,RIDGELAND,MS,...,999 Other,5411 Legal Services,5411.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.15197,NaN
2,516301,516301,5587558700,4/2/21,470,PPP,MISSISSIPPI METHODIST SENIOR SERVICES IN,109 S Broadway St,Tupelo,MS,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,NaN
3,516302,516302,8446217010,4/8/20,470,PPP,SOUTHERN ELECTRIC CORPORATION OF MISSISSIPPI,4374-A MANGUM DR,FLOWOOD,MS,...,999 Other,2371 Utility System Construction,2371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"4374-A MANGUM DR, FLOWOOD, MS",32.31944,-90.10765,NaN
4,516303,516303,3918387109,4/12/20,470,PPP,FRANKLIN CORPORATION,600 FRANKLIN DR,HOUSTON,MS,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.01671,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5497,521796,521796,6662018610,3/23/21,470,PPS,GATHRIGHT-REED MEDICAL SUPPLY,300 Heritage Dr,Oxford,MS,...,999 Other,4234 Professional and Commercial Equipment and...,4234.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"300 Heritage Dr, Oxford, MS",34.367306,-89.56031,NaN
5498,521797,521797,7192318409,2/11/21,470,PPS,FULWOOD CONSTRUCTION COMPANY LLC,6229 Highway 305 N Ste O,Olive Branch,MS,...,999 Other,5324 Commercial and Industrial Machinery and E...,5324.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6229 Highway 305 N Ste O, Olive Branch, MS",34.95232,-89.830414,NaN
5499,521798,521798,7483017308,4/30/20,470,PPP,"S & J TRANSFER & STORAGE, LTD",41033 OLD HIGHWAY 45 South,ABERDEEN,MS,...,999 Other,4931 Warehousing and Storage,4931.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",None,None,NaN
5500,521799,521799,7726238600,3/24/21,470,PPS,OTIS LOGGING INC.,116 Price St,McComb,MS,...,998 All Other Miscellaneous Manufacturing,1133 Logging,1133.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"116 Price St, McComb, MS",31.24796,-90.46982,NaN


In [20]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoMS = ppp_togeoMS[cols]
ppp_togeoMS.head()

,LoanNumber,FIPS_z,Lat,Long
0,3506337207,280470014022015,30.381123,-89.044
1,8685327005,280890302051012,32.446636,-90.15197
2,5587558700,280819507001030,34.257282,-88.705376
3,8446217010,281210203041000,32.31944,-90.10765
4,3918387109,280179503002008,33.914646,-89.01671


In [21]:
ppp_geo = ppp_geo.merge(ppp_togeoMS_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat_x,Long_x,FIPS_z_x,FIPS_z_y,Lat_y,Long_y
0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,29150.0,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,NaN,NaN,NaN
1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,29420.0,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,NaN,NaN,NaN
2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,98112.0,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,NaN,NaN,NaN
3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,92407.0,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN,NaN,NaN,NaN
4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,NaN,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,NaN,NaN,NaN


In [22]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,29150.0,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,29420.0,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,98112.0,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,92407.0,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,NaN,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [53]:
ppp_geo = ppp_geo.drop_duplicates(subset=["LoanNumber"])
ppp_geo

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,29150.0,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,29420.0,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,98112.0,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,92407.0,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,NaN,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978144,977196,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,WY,82601.0,...,999 Other,6212 Offices of Dentists,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",NaN,NaN,NaN
978145,977197,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,WY,82935.0,...,999 Other,6244 Child Day Care Services,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",NaN,NaN,NaN
978146,977198,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,WY,82727.0,...,999 Other,2382 Building Equipment Contractors,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",NaN,NaN,NaN
978147,977199,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,WY,82801.0,...,999 Other,6216 Home Health Care Services,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",NaN,NaN,NaN


In [ ]:
ppp_geo["Lat"].replace('None', np.nan, inplace=True)

In [55]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [58]:
ppp_geo_ms = ppp_geo.loc[ppp_geo["BorrowerState"] =="MS"]
ppp_geo_ms

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
516299,516299,3506337207,4/27/20,470,PPP,STAFF PRO LLC,238 COURTHOUSE RD,GULFPORT,MS,39507.0,...,999 Other,5613 Employment Services,5613.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044,280470014022015
516300,516300,8685327005,4/8/20,470,PPP,"BUTLER SNOW, LLP",1020 HIGHLAND COLONY PKWY,RIDGELAND,MS,39157.0,...,999 Other,5411 Legal Services,5411.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.15197,280890302051012
516301,516301,5587558700,4/2/21,470,PPP,MISSISSIPPI METHODIST SENIOR SERVICES IN,109 S Broadway St,Tupelo,MS,38804.0,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,280819507001030
516302,516302,8446217010,4/8/20,470,PPP,SOUTHERN ELECTRIC CORPORATION OF MISSISSIPPI,4374-A MANGUM DR,FLOWOOD,MS,39232.0,...,999 Other,2371 Utility System Construction,2371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"4374-A MANGUM DR, FLOWOOD, MS",32.31944,-90.10765,281210203041000
516303,516303,3918387109,4/12/20,470,PPP,FRANKLIN CORPORATION,600 FRANKLIN DR,HOUSTON,MS,38851.0,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.01671,280179503002008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522744,521796,6662018610,3/23/21,470,PPS,GATHRIGHT-REED MEDICAL SUPPLY,300 Heritage Dr,Oxford,MS,38655.0,...,999 Other,4234 Professional and Commercial Equipment and...,4234.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"300 Heritage Dr, Oxford, MS",34.367306,-89.56031,280719503042000
522745,521797,7192318409,2/11/21,470,PPS,FULWOOD CONSTRUCTION COMPANY LLC,6229 Highway 305 N Ste O,Olive Branch,MS,38654.0,...,999 Other,5324 Commercial and Industrial Machinery and E...,5324.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6229 Highway 305 N Ste O, Olive Branch, MS",34.95232,-89.830414,280330708213005
522746,521798,7483017308,4/30/20,470,PPP,"S & J TRANSFER & STORAGE, LTD",41033 OLD HIGHWAY 45 South,ABERDEEN,MS,39730.0,...,999 Other,4931 Warehousing and Storage,4931.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",None,NaN,NaN
522747,521799,7726238600,3/24/21,470,PPS,OTIS LOGGING INC.,116 Price St,McComb,MS,39648.0,...,998 All Other Miscellaneous Manufacturing,1133 Logging,1133.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"116 Price St, McComb, MS",31.24796,-90.46982,281139504002077


In [61]:
null_check = ppp_geo_ms.loc[ppp_geo_ms["FIPS_z"].isnull()]
null_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
516306,516304,8848687101,4/15/20,470,PPP,"NEEL SCHAFFER, INC",125 South Congress St. Suite 1100,JACKSON,MS,39201.0,...,999 Other,2379 Other Heavy and Civil Engineering Constru...,2379.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"125 South Congress St. Suite 1100, JACKSON, MS",None,NaN,NaN
516320,516310,3239977107,4/11/20,470,PPP,SOUTHERN TIRE MART LLC,800 HIGHWAY 98 BYP,COLUMBIA,MS,39429.0,...,999 Other,3262 Rubber Product Manufacturing,3262.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"800 HIGHWAY 98 BYP, COLUMBIA, MS",None,NaN,NaN
516321,516311,4931077000,4/4/20,470,PPP,PUCKETT MACHINERY COMPANY,100 CATERPILLAR DR,FLOWOOD,MS,39232.0,...,999 Other,8113 Commercial and Industrial Machinery and E...,8113.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"100 CATERPILLAR DR, FLOWOOD, MS",None,NaN,NaN
516323,516313,7549477005,4/7/20,470,PPP,B & B ELECTRICAL & UTILITY CONTRACTORS,530 SOLDIER COLONY RD,CANTON,MS,39046.0,...,999 Other,2371 Utility System Construction,2371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,250 to 499,"530 SOLDIER COLONY RD, CANTON, MS",None,NaN,NaN
516324,516314,4060597105,4/12/20,470,PPP,"MAX HOME, LLC",101 MAX PL,FULTON,MS,38843.0,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,MISSISSIPPI DISTRICT OFFICE,e $5-10 million,500 or more,"101 MAX PL, FULTON, MS",None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522732,521785,1384498403,2/1/21,470,PPP,ACE HOME INC,100 Williamsburg Cv,Oxford,MS,38655.0,...,999 Other,2361 Residential Building Construction,2361.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"100 Williamsburg Cv, Oxford, MS",None,NaN,NaN
522733,521786,2132888302,1/20/21,470,PPS,PAGE CONSTRUCTION LLC,10674 Brownsferry Rd,Sarah,MS,38665.0,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"10674 Brownsferry Rd, Sarah, MS",None,NaN,NaN
522741,521793,5005907006,4/4/20,470,PPP,"JRELLIS HOLDINGS, LLC",PO BOX 117,BOONEVILLE,MS,38829.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"PO BOX 117, BOONEVILLE, MS",None,NaN,NaN
522743,521795,5575347104,4/13/20,470,PPP,"MISSISSIPPI HEALTH PARTNERS, INC",1501 LAKELAND DR STE 200,JACKSON,MS,39216.0,...,999 Other,"5242 Agencies, Brokerages, and Other Insurance...",5242.0,MISSISSIPPI DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1501 LAKELAND DR STE 200, JACKSON, MS",None,NaN,NaN
